In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
 

 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

sns.set(style='white', context='notebook', palette='deep')
df_train = pd.read_csv(r"/content/drive/MyDrive/datasetProcessed.csv")
# df_test = pd.read_csv(r"C:\Users\Nauman Rajput\Desktop\archive_2\credit_train.csv")

df=df_train.copy()

In [ ]:
df.head()


,ts,src_port,dst_port,src_ip,dst_ip,proto,conn_state,label
0,1556076559,40461,4111,192.168.1.32,192.168.1.180,tcp,S0,1
1,1556027136,52242,12036,192.168.1.30,192.168.1.180,tcp,S0,1
2,1556027653,48983,41511,192.168.1.32,192.168.1.195,tcp,S0,1
3,1556077155,6156,56643,192.168.1.79,192.168.1.31,tcp,RSTOS0,1
4,1556027666,61281,1072,192.168.1.32,192.168.1.195,tcp,S0,1


In [ ]:
statistics_of_data = []
for col in df.columns:
  statistics_of_data.append((col,
                             df[col].nunique(),
                             df[col].isnull().sum()*100/df.shape[0],
                             df[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                             df[col].dtype
                             ))
stats_df = pd.DataFrame(statistics_of_data, columns=['Feature', 'Uniq_val', 'missing_val', 'val_biggest_cat', 'type'])

In [ ]:
stats_df

,Feature,Uniq_val,missing_val,val_biggest_cat,type
0,ts,155692,0.0,0.059817,int64
1,src_port,49346,0.0,3.959618,int64
2,dst_port,17511,0.0,21.061567,int64
3,src_ip,5563,0.0,11.092150,object
4,dst_ip,4297,0.0,17.747029,object
5,proto,3,0.0,72.521354,object
6,conn_state,13,0.0,29.993285,object
7,label,2,0.0,52.779268,int64


In [ ]:
import numpy as np

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(df):
    """create a dataframe and optimize its memory usage"""
    df = reduce_mem_usage(df)
    return df
df=import_data(df) 

Memory usage of dataframe is 20.82 MB
Memory usage after optimization is: 6.51 MB
Decreased by 68.7%


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341043 entries, 0 to 341042
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   ts          341043 non-null  int32   
 1   src_port    341043 non-null  int32   
 2   dst_port    341043 non-null  int32   
 3   src_ip      341043 non-null  category
 4   dst_ip      341043 non-null  category
 5   proto       341043 non-null  category
 6   conn_state  341043 non-null  category
 7   label       341043 non-null  int8    
dtypes: category(4), int32(3), int8(1)
memory usage: 6.5 MB


In [ ]:
cat_col = df.select_dtypes(include='category')
cat_col

,src_ip,dst_ip,proto,conn_state
0,192.168.1.32,192.168.1.180,tcp,S0
1,192.168.1.30,192.168.1.180,tcp,S0
2,192.168.1.32,192.168.1.195,tcp,S0
3,192.168.1.79,192.168.1.31,tcp,RSTOS0
4,192.168.1.32,192.168.1.195,tcp,S0
...,...,...,...,...
341038,192.168.1.190,91.189.95.3,udp,SF
341039,192.168.1.180,192.168.1.190,udp,SF
341040,192.168.1.190,192.168.1.190,tcp,OTH
341041,172.17.0.5,192.168.1.190,udp,SHR


In [ ]:
index_list=[]
for index,val in enumerate(df['src_ip'].str.split('.')):
  try:
    m = int(val[0])
  except:
    index_list.append(index)


  

In [ ]:
index_list

[161085,
 161174,
 161187,
 161201,
 161204,
 161254,
 161289,
 161297,
 161365,
 161387,
 161497,
 161505,
 161521,
 161600,
 161610,
 161652,
 161775,
 161793,
 161829,
 161831,
 161844,
 161858,
 161953,
 161981,
 162047,
 162052,
 162177,
 162257,
 162275,
 162295,
 162425,
 162432,
 162530,
 162608,
 162693,
 162757,
 162763,
 162805,
 162855,
 162897,
 162904,
 162911,
 162958,
 163076,
 163116,
 163284,
 163319,
 163343,
 163403,
 163437,
 163447,
 163469,
 163502,
 163552,
 163570,
 163604,
 163653,
 163662,
 163702,
 163733,
 163864,
 163899,
 163999,
 164013,
 164181,
 164197,
 164321,
 164338,
 164350,
 164414,
 164555,
 164565,
 164571,
 164582,
 164603,
 164803,
 164805,
 164831,
 164966,
 164980,
 165057,
 165111,
 165119,
 165144,
 165151,
 165222,
 165282,
 165306,
 165324,
 165380,
 165404,
 165443,
 165515,
 165574,
 165595,
 165710,
 165788,
 165799,
 165939,
 165943,
 165952,
 166026,
 166034,
 166036,
 166173,
 166312,
 166498,
 166698,
 166823,
 166862,
 166946,
 

In [ ]:
df.iloc[index_list,:]

,ts,src_port,dst_port,src_ip,dst_ip,proto,conn_state,label
161085,1554270005,135,136,fe80::a691:b1ff:fe1e:5790,fe80::20c:29ff:feee:e07a,icmp,OTH,0
161174,1556424119,135,136,fe80::a691:b1ff:fe1e:5790,fe80::bc7f:2fed:bb00:6bf2,icmp,OTH,0
161187,1554269295,59950,5355,fe80::e4a6:b91f:48f3:ac01,ff02::1:3,udp,S0,0
161201,1554305649,136,135,2405:6e00:10ce:2c00:8520:4eba:6770:702a,fe80::a691:b1ff:fe1e:5790,icmp,OTH,0
161204,1554298202,143,0,fe80::f554:8774:9968:b60b,ff02::16,icmp,OTH,0
...,...,...,...,...,...,...,...,...
340859,1556431676,49795,1900,fe80::2020:3916:4b74:9155,ff02::c,udp,S0,0
340900,1556427956,5353,5353,fe80::b0bd:4998:5e18:82ce,ff02::fb,udp,S0,0
340904,1556427951,135,136,fe80::a691:b1ff:fe1e:5790,fe80::a58c:cd03:5237:3b49,icmp,OTH,0
340991,1554271884,5353,5353,2405:6e00:10ce:2c00:c058:dbaa:f212:a2d8,ff02::fb,udp,S0,0


In [ ]:
df.shape

(341043, 8)

In [ ]:
df.drop(index_list,axis=0,inplace=True)

In [ ]:
df.shape


(337265, 8)

Dealing with Cat columns

In [ ]:
cat_col = df.select_dtypes(include='category')
cat_col

,src_ip,dst_ip,proto,conn_state
0,192.168.1.32,192.168.1.180,tcp,S0
1,192.168.1.30,192.168.1.180,tcp,S0
2,192.168.1.32,192.168.1.195,tcp,S0
3,192.168.1.79,192.168.1.31,tcp,RSTOS0
4,192.168.1.32,192.168.1.195,tcp,S0
...,...,...,...,...
341038,192.168.1.190,91.189.95.3,udp,SF
341039,192.168.1.180,192.168.1.190,udp,SF
341040,192.168.1.190,192.168.1.190,tcp,OTH
341041,172.17.0.5,192.168.1.190,udp,SHR


In [ ]:
# df1 = df[df['src_ip'].str.contains('172.17.') | (df['src_ip'].str.contains('192.168.'))]

In [ ]:
df.shape

(337265, 8)

In [ ]:
# len(df1['src_ip'].value_counts().values)

In [ ]:
d = dict(df['src_ip'].value_counts().sort_values(ascending=False)[:30])

In [ ]:
df1 = df[df['src_ip'].isin(d.keys())]


In [ ]:
d = dict(df1['dst_ip'].value_counts().sort_values(ascending=False)[:50])
d

{'192.168.1.190': 59031,
 '192.168.1.195': 37865,
 '192.168.1.152': 37494,
 '192.168.1.193': 27862,
 '192.168.1.184': 20631,
 '192.168.1.33': 18451,
 '192.168.1.1': 17353,
 '192.168.1.79': 12287,
 '192.168.1.194': 8354,
 '239.255.255.250': 8014,
 '192.168.1.255': 7886,
 '127.0.0.1': 7408,
 '192.168.1.37': 4062,
 '192.168.1.180': 3670,
 '176.28.50.165': 3562,
 '52.59.177.21': 2712,
 '224.0.0.251': 2441,
 '18.184.104.180': 2394,
 '224.0.0.252': 2212,
 '192.168.1.169': 1927,
 '52.28.231.150': 1512,
 '203.119.86.101': 1491,
 '18.194.169.124': 1437,
 '192.0.32.59': 1228,
 '204.61.216.50': 1017,
 '192.168.1.186': 884,
 '8.8.8.8': 841,
 '192.5.6.30': 830,
 '192.168.1.46': 704,
 '203.119.95.53': 545,
 '192.33.14.30': 530,
 '202.12.31.53': 469,
 '192.168.1.49': 468,
 '192.168.1.32': 357,
 '64.124.14.21': 355,
 '199.7.83.42': 349,
 '192.26.92.30': 342,
 '193.108.88.128': 341,
 '192.58.128.30': 332,
 '192.5.5.241': 324,
 '192.168.1.31': 324,
 '192.168.1.250': 313,
 '204.61.216.100': 300,
 '192.30

In [ ]:
df2 = df1[df1['dst_ip'].isin(d.keys())]


In [ ]:
df2

,ts,src_port,dst_port,src_ip,dst_ip,proto,conn_state,label
0,1556076559,40461,4111,192.168.1.32,192.168.1.180,tcp,S0,1
1,1556027136,52242,12036,192.168.1.30,192.168.1.180,tcp,S0,1
2,1556027653,48983,41511,192.168.1.32,192.168.1.195,tcp,S0,1
3,1556077155,6156,56643,192.168.1.79,192.168.1.31,tcp,RSTOS0,1
4,1556027666,61281,1072,192.168.1.32,192.168.1.195,tcp,S0,1
...,...,...,...,...,...,...,...,...
341037,1556288476,61171,41952,192.168.1.195,192.168.1.1,tcp,SF,0
341039,1556079253,46657,53,192.168.1.180,192.168.1.190,udp,SF,0
341040,1554268119,43530,7878,192.168.1.190,192.168.1.190,tcp,OTH,0
341041,1556026367,34772,53,172.17.0.5,192.168.1.190,udp,SHR,0


Ordinal Encoding in Proto


In [ ]:
df2.proto.value_counts()

tcp     233087
udp      69373
icmp       248
Name: proto, dtype: int64

In [ ]:
mapping = {
    "tcp":2,
"udp":1,
"icmp":0
}
df2["proto"] = df2["proto"].replace(mapping)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df2

,ts,src_port,dst_port,src_ip,dst_ip,proto,conn_state,label
0,1556076559,40461,4111,192.168.1.32,192.168.1.180,2,S0,1
1,1556027136,52242,12036,192.168.1.30,192.168.1.180,2,S0,1
2,1556027653,48983,41511,192.168.1.32,192.168.1.195,2,S0,1
3,1556077155,6156,56643,192.168.1.79,192.168.1.31,2,RSTOS0,1
4,1556027666,61281,1072,192.168.1.32,192.168.1.195,2,S0,1
...,...,...,...,...,...,...,...,...
341037,1556288476,61171,41952,192.168.1.195,192.168.1.1,2,SF,0
341039,1556079253,46657,53,192.168.1.180,192.168.1.190,1,SF,0
341040,1554268119,43530,7878,192.168.1.190,192.168.1.190,2,OTH,0
341041,1556026367,34772,53,172.17.0.5,192.168.1.190,1,SHR,0


In [ ]:

y = pd.get_dummies(df2.conn_state, prefix='State')

In [ ]:
y.drop("State_OTH",axis=1,inplace=True)

In [ ]:
merged = pd.concat([df2,y],axis='columns')

In [ ]:
merged.drop("conn_state",axis=1,inplace=True)

In [ ]:
merged

,ts,src_port,dst_port,src_ip,dst_ip,proto,label,State_REJ,State_RSTO,State_RSTOS0,State_RSTR,State_RSTRH,State_S0,State_S1,State_S2,State_S3,State_SF,State_SH,State_SHR
0,1556076559,40461,4111,192.168.1.32,192.168.1.180,2,1,0,0,0,0,0,1,0,0,0,0,0,0
1,1556027136,52242,12036,192.168.1.30,192.168.1.180,2,1,0,0,0,0,0,1,0,0,0,0,0,0
2,1556027653,48983,41511,192.168.1.32,192.168.1.195,2,1,0,0,0,0,0,1,0,0,0,0,0,0
3,1556077155,6156,56643,192.168.1.79,192.168.1.31,2,1,0,0,1,0,0,0,0,0,0,0,0,0
4,1556027666,61281,1072,192.168.1.32,192.168.1.195,2,1,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341037,1556288476,61171,41952,192.168.1.195,192.168.1.1,2,0,0,0,0,0,0,0,0,0,0,1,0,0
341039,1556079253,46657,53,192.168.1.180,192.168.1.190,1,0,0,0,0,0,0,0,0,0,0,1,0,0
341040,1554268119,43530,7878,192.168.1.190,192.168.1.190,2,0,0,0,0,0,0,0,0,0,0,0,0,0
341041,1556026367,34772,53,172.17.0.5,192.168.1.190,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
merged['src_ip']= LabelEncoder().fit_transform(merged['src_ip'])

In [ ]:
merged['dst_ip']= LabelEncoder().fit_transform(merged['dst_ip'])


In [ ]:
# df_final = merged[['src_ip', 'dst_ip', 'proto',
#        'State_REJ', 'State_RSTO', 'State_RSTOS0', 'State_RSTR', 'State_RSTRH',
#        'State_S0', 'State_S1', 'State_S2', 'State_S3', 'State_SF', 'State_SH',
#        'State_SHR','label']]

In [ ]:
df_final = merged.copy()

In [ ]:
df_final

,ts,src_port,dst_port,src_ip,dst_ip,proto,label,State_REJ,State_RSTO,State_RSTOS0,State_RSTR,State_RSTRH,State_S0,State_S1,State_S2,State_S3,State_SF,State_SH,State_SHR
0,1556076559,40461,4111,20,9,2,1,0,0,0,0,0,1,0,0,0,0,0,0
1,1556027136,52242,12036,18,9,2,1,0,0,0,0,0,1,0,0,0,0,0,0
2,1556027653,48983,41511,20,15,2,1,0,0,0,0,0,1,0,0,0,0,0,0
3,1556077155,6156,56643,29,18,2,1,0,0,1,0,0,0,0,0,0,0,0,0
4,1556027666,61281,1072,20,15,2,1,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341037,1556288476,61171,41952,16,6,2,0,0,0,0,0,0,0,0,0,0,1,0,0
341039,1556079253,46657,53,11,12,1,0,0,0,0,0,0,0,0,0,0,1,0,0
341040,1554268119,43530,7878,13,12,2,0,0,0,0,0,0,0,0,0,0,0,0,0
341041,1556026367,34772,53,2,12,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
X =df_final.drop('label',axis=1)
Y = df_final.label.values
Y

array([1, 1, 1, ..., 0, 0, 0], dtype=int8)

In [ ]:
from sklearn.preprocessing import StandardScaler
# X = StandardScaler().fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split 


x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30)
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
# rfc.score(x_train, y_train)
y_predict = rfc.predict(x_test)

In [ ]:
print(y_predict)
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, y_predict))
print(classification_report(y_test, y_predict))

[1 0 0 ... 0 1 1]
[[43487     0]
 [    2 47324]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     43487
           1       1.00      1.00      1.00     47326

    accuracy                           1.00     90813
   macro avg       1.00      1.00      1.00     90813
weighted avg       1.00      1.00      1.00     90813



In [ ]:
df_final.label.value_counts()

1    157888
0    144820
Name: label, dtype: int64

Decision Tree


In [ ]:
from sklearn import tree

# Create a tree object
classifier = tree.DecisionTreeClassifier()
# Ask the tree to fit the data to the labels!  That's it!
classifier = classifier.fit(x_train, y_train)

from sklearn.metrics import confusion_matrix
results = confusion_matrix(y_test, classifier.predict(x_test))

In [ ]:
results

array([[43484,     3],
       [    2, 47324]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, classifier.predict(x_test))

0.9999449418034863

Decision Tree

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
dt = cross_val_score(tree.DecisionTreeClassifier(),X,Y,cv=5).mean()

Random Forest Classifier

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
rf = cross_val_score(RandomForestClassifier(n_estimators=100),X,Y,cv=5).mean()

SVC

In [ ]:
from sklearn.svm import SVC
svc = cross_val_score(SVC(kernel = 'rbf', random_state = 0),X,Y,cv=5).mean()


Logisitic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = cross_val_score(LogisticRegression(random_state = 0),X,Y,cv=5)

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = cross_val_score(KNeighborsClassifier(n_neighbors = 3),X,Y,cv=5)


Gaussian Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
gn = cross_val_score( GaussianNB(),X,Y,cv=5)


In [ ]:
model_ev = pd.DataFrame({'Model': ['Logistic Regression','Naive Bayes','Random Forest',
                    'K-Nearest Neighbour','Decision Tree','Support Vector Machine'], 'Accuracy': [lr ,
                    gn,rf,knn,dt ,svc ]})
model_ev

In [ ]:
colors = ['red','green','blue','purple','yellow','orange',]
plt.figure(figsize=(12,5))
plt.title("barplot Represent Accuracy of different models")
plt.xlabel("Accuracy % \n")
plt.ylabel("Algorithms \n")
plt.bar(model_ev['Model'],model_ev['Accuracy'],color = colors)
plt.show()